This notebook collects query time, recall, and memory use in the hierarchical navigable small world index solution

In [8]:
import pickle
import collections
import matplotlib.pyplot as plt
import faiss
import numpy as np
import itertools
import statistics
import matplotlib.cm as cm
import time
import os
from time import perf_counter_ns

In [9]:
# read in embeddings and cluster label info
with open('embedding64.pickle', 'rb') as fp:
    embedding64 = pickle.load(fp)
with open('label_info.pickle', 'rb') as fp:
    label_info = pickle.load(fp)

print(embedding64.keys())
print(label_info.keys())

# create helpful dicts for the cluster labels
# also look at n counts for the clusters
major_labels = list(set(label_info['cluster label']))
minor_labels = list(set(label_info['cluster label minor']))

major = dict(zip(major_labels,[[] for item in major_labels]))
minor = dict(zip(minor_labels,[[] for item in minor_labels]))

true_major = label_info['cluster label']
true_minor = label_info['cluster label minor']

for j in range(len(label_info['cluster label'])):
    maj = label_info['cluster label'][j]
    min = label_info['cluster label minor'][j]

    if maj in major.keys():
        major[maj].append(j)
    if min in minor.keys():
        minor[min].append(j)

dict_keys(['embed_all', 'embed_raw', 'embed_l2_norm', 'restore_order', 'embed_correct_coverage_fh', 'embed_l2_norm_correct_coverage_fh'])
dict_keys(['batch id', 'age', 'total_cg', 'average_cg_rate', 'total_ch', 'average_ch_rate', 'hic_counts', 'cell_name_higashi', 'major', 'minor', 'cluster label', 'cluster label minor'])


In [10]:
# function to get memory footprint for index
# source: https://www.pinecone.io/learn/series/faiss/product-quantization/
def get_memory(index):
    faiss.write_index(index,'./temp.index')
    file_size = os.path.getsize('./temp.index')
    os.remove('./temp.index')
    return file_size
    
# function to calculate recall based on search results
def get_recall_min(i, result_min):
    # recall: TP / cluster size
    return len(set(minor[true_minor[i]]).intersection(result_min)) / len(minor[true_minor[i]])

def get_recall_maj(i, result_maj):
    # recall: TP / cluster size
    return len(set(major[true_major[i]]).intersection(result_maj)) / len(major[true_major[i]])

In [11]:
# create input database
database = np.array(embedding64["embed_l2_norm"]) 

In [12]:
def query_rep(index, query, k):
    start1 = perf_counter_ns()
    for x in range(100):
        D, I = index.search(query, k)
    end1 = perf_counter_ns()
        
    start2 = perf_counter_ns()
    for x in range(100):
        D, I = index.search(query, k)
    end2 = perf_counter_ns()
        
    start3 = perf_counter_ns()
    for x in range(100):
        D, I = index.search(query, k)
    end3 = perf_counter_ns()

    times = [(end1-start1),(end2-start2),(end3-start3)]
    times.sort()

    return I, times

In [15]:
# define experiment function
def HNSW_experiment(index_str):
    # initialize empty arrays for results
    recall_min = np.zeros([4238])
    recall_maj = np.zeros([4238])
    speed_min = np.zeros([4238])
    speed_maj = np.zeros([4238])
    memory = np.zeros([4238])
    
    for i in range(4238):
        # subset dataset
        db_subset = np.delete(database, i, 0)
        query = np.array([database[i]])
        k_major = len(major[true_major[i]]) # size of true cluster (how many neighbors to return)
        k_minor = len(minor[true_minor[i]]) # size of true cluster (how many neighbors to return)
        
        # create and train index
        index = faiss.index_factory(64, index_str)
        index.train(db_subset)
        index.add(database)

        # how much memory is used?
        memory[i] = get_memory(index)
       
        # major cluster query
        I, time = query_rep(index, query, k_major)
        recall_maj[i] = get_recall_maj(i, I[0])
        speed_maj[i] = time[0]

        # minor cluster query
        I, time = query_rep(index, query, k_minor)
        recall_min[i] = get_recall_min(i, I[0])
        speed_min[i] = time[0]        

    # return all results
    return recall_min, recall_maj, speed_min, speed_maj, memory

In [16]:
### EXPERIMENT A: M=16
recall_min_A, recall_maj_A, speed_min_A, speed_maj_A, memory_A = HNSW_experiment("HNSW16,Flat")

with open('HNSW/ExperimentA.npy', 'wb') as f:
    np.save(f, recall_min_A)
    np.save(f, recall_maj_A)
    np.save(f, speed_min_A)
    np.save(f, speed_maj_A)
    np.save(f, memory_A)

# with open('HNSW/ExperimentA.npy', 'rb') as f:
#     recall_min_A = np.load(f)
#     recall_maj_A = np.load(f)
#     speed_min_A = np.load(f)
#     speed_maj_A = np.load(f)
#     memory_A = np.load(f)

In [17]:
### EXPERIMENT B: M=32
recall_min_B, recall_maj_B, speed_min_B, speed_maj_B, memory_B = HNSW_experiment("HNSW32,Flat")

with open('HNSW/ExperimentB.npy', 'wb') as f:
    np.save(f, recall_min_B)
    np.save(f, recall_maj_B)
    np.save(f, speed_min_B)
    np.save(f, speed_maj_B)
    np.save(f, memory_B)

# with open('HNSW/ExperimentB.npy', 'rb') as f:
#     recall_min_B = np.load(f)
#     recall_maj_B = np.load(f)
#     speed_min_B = np.load(f)
#     speed_maj_B = np.load(f)
#     memory_B = np.load(f)

In [18]:
### EXPERIMENT C: M=64
recall_min_C, recall_maj_C, speed_min_C, speed_maj_C, memory_C = HNSW_experiment("HNSW64,Flat")

with open('HNSW/ExperimentC.npy', 'wb') as f:
    np.save(f, recall_min_C)
    np.save(f, recall_maj_C)
    np.save(f, speed_min_C)
    np.save(f, speed_maj_C)
    np.save(f, memory_C)

# with open('HNSW/ExperimentC.npy', 'rb') as f:
#     recall_min_C = np.load(f)
#     recall_maj_C = np.load(f)
#     speed_min_C = np.load(f)
#     speed_maj_C = np.load(f)
#     memory_C = np.load(f)

In [20]:
### EXPERIMENT D: M=128
recall_min_D, recall_maj_D, speed_min_D, speed_maj_D, memory_D = HNSW_experiment("HNSW128,Flat")

with open('HNSW/ExperimentD.npy', 'wb') as f:
    np.save(f, recall_min_D)
    np.save(f, recall_maj_D)
    np.save(f, speed_min_D)
    np.save(f, speed_maj_D)
    np.save(f, memory_D)

# with open('HNSW/ExperimentD.npy', 'rb') as f:
#     recall_min_D = np.load(f)
#     recall_maj_D = np.load(f)
#     speed_min_D = np.load(f)
#     speed_maj_D = np.load(f)
#     memory_D = np.load(f)